# 1 Mbox reader

Lecture d'un fichier mbox (mailbox) et export d’une sélection de données en csv (comma separated values).

On s'intéresse aux expéditeurs, destinataires, sujets des e-mails et heures d'envoi.

Le but: une analyse rapide et subjective pour mieux comprendre le sentiment d'être écrasé par des e-mails non pertinents.

### Hypothèses:
* Groupes de distribution trop larges
* Mis en copie trop fréquemment
* Communiqués diffusés trop largement (au-delà des pays / régions linguistiques)

### Marche à suivre
#### Ce notebook
1. extraire expéditeurs, destinataires, sujets et heures d'envoi
2. sélectionner une période de 31 jours

#### Notebook «2 email rater»
1. départager les e-mails: inbox, sent, spam, etc.
2. résultats en chiffres
3. graphiques par heure

In [52]:
import mailbox
from email.header import Header, decode_header, make_header
import pandas as pd

In [53]:
%%time
mbox = mailbox.mbox('data/gmail.mbox')

CPU times: user 202 µs, sys: 158 µs, total: 360 µs
Wall time: 282 µs


In [54]:
# Pour décoder les champs de chaque e-mail
def decode(value):
    if not isinstance(value, str):
        return value
    return str(make_header(decode_header(value))).replace('\n', '')

In [55]:
%%time
i = 0
message_list = []
for message in mbox:
    #print(message['From'])
    message_list.append({
        'From': decode(message['From']),
        'Subject': decode(message['Subject']),
        'To': decode(message['To']),
        'Cc': decode(message['Cc']),
        'Date': message['Date'],
        'Label': decode(message['X-Gmail-Labels']),
        'Spam': message['X-SPAM-FLAG']
    })
    
    # Ecrit un point chaque 100 e-mail. Pour voir que ça avance
    i += 1
    if i > 99:
        print('.', end='')
        i = 0

.....................................................................................................................................................................................................................................................CPU times: user 3min 46s, sys: 25.7 s, total: 4min 12s
Wall time: 4min 16s


In [56]:
# On crée un tableau des données à retenir: expéditeurs, destinataires, sujets des e-mails, étiquettes, heures d'envoi
df = pd.DataFrame(message_list)
df.shape

(24550, 7)

In [57]:
# On regarde les 5 premières lignes (mis en commentaire pour masquer les adresses des expéditeurs)

# df.head()

In [58]:
df['To'] = df['To'].apply(lambda x: str(x))

In [59]:
# Nombre d'e-mails sans objet
len(df[df['Subject'].isnull()])

273

In [60]:
# On transforme les champs vides en texte pour permettre
# la recherche textuelle avec str.contains() (prochain notebook)
df.fillna('', inplace=True)

In [61]:
# E-mails sans date?
df[df.Date.isnull()]

,Cc,Date,From,Label,Spam,Subject,To


In [62]:
df[df['Label'].str.contains('Messagerie instantanée')].head()

,Cc,Date,From,Label,Spam,Subject,To
901,,,"""Ronga, Paul"" <paul.ronga@tdg.ch>",Messagerie instantanée,,,None
995,,,Cecile Denayrouse <cecile.denayrouse@tdg.ch>,"Messagerie instantanée,A moi",,,None
996,,,Cecile Denayrouse <cecile.denayrouse@tdg.ch>,"Messagerie instantanée,A moi",,,None
997,,,Cecile Denayrouse <cecile.denayrouse@tdg.ch>,"Messagerie instantanée,A moi",,,None
998,,,Cecile Denayrouse <cecile.denayrouse@tdg.ch>,"Messagerie instantanée,A moi",,,None


In [63]:
# La messagerie instantanée fonctionne différemment (pas de date). On l'enlève
df = df[-df['Label'].str.contains('Messagerie instantanée')].copy()

In [64]:
# Avant la lecture des datetime, on ne garde que 2018
df = df[df.Date.str.contains('2018')].copy()

In [65]:
# Lecture des datetime
df['Timestamp'] = pd.to_datetime(df.Date, errors='coerce')

In [66]:
# Quelles dates n’ont pas été lues? Un spam
df[df['Timestamp'] != df['Timestamp']]

,Cc,Date,From,Label,Spam,Subject,To,Timestamp
775,,"Fri, 02 Feb 2018 27:20:07 +0600",<eskat@adm.unifi.it>,"Non lus,A moi,Spam",,Es ist endlich HIER,<paul.ronga@tdg.ch>,NaT


In [67]:
# On ne garde que les dates correctement lues
df = df[df['Timestamp'] == df['Timestamp']].copy()

In [68]:
df.index = df['Timestamp']

In [69]:
# Combien d’e-mails reçus par mois? (NB: le drapeau "Spam"
# ne contient pas tout le dossier spam, qui compte des centaines d'e-mails)
df.resample('M').count()

,Cc,Date,From,Label,Spam,Subject,To,Timestamp
Timestamp,,,,,,,,
2018-01-31,2983,2983,2983,2983,2983,2983,2983,2983
2018-02-28,2228,2228,2228,2228,2228,2228,2228,2228


In [70]:
# Nombre d'e-mails en 2018, au total
df.shape

(5211, 8)

In [71]:
# On enlève les sauts de lignes des expéditeurs, destinataires, objets
def removeLineBreaks(value):
    if not isinstance(value, str):
        return value
    return value.replace('\r', '').replace('\n', '')
df['To'] = df['To'].apply(removeLineBreaks)
df['From'] = df['From'].apply(removeLineBreaks)
df['Subject'] = df['Subject'].apply(removeLineBreaks)

In [72]:
# Sauvegardarde du corpus 2018 complet
df.to_csv('data/corpus_2018.csv')

In [73]:
# Nombre d'e-mails du 14 janvier au 14 février
df['2018-01-14':'2018-02-14'].shape

(4543, 8)

In [74]:
dfx = df['2018-01-14':'2018-02-14'].copy()

In [75]:
sent = dfx.Label.str.contains('Messages envoyés')

In [76]:
# Taille des messages envoyés vs de tout le reste
dfx[sent].shape, dfx[-sent].shape

((345, 8), (4198, 8))

In [77]:
del dfx['Timestamp']

In [78]:
# On enregistre les messages reçus / envoyés du 14 janvier au 14 février
dfx[-sent].to_csv('data/received.csv')
dfx[sent].to_csv('data/sent.csv')